<a href="https://colab.research.google.com/github/tanynova99/thesis_materials/blob/main/corpus_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')

import re
import pprint

import nltk
from nltk import word_tokenize
from nltk.corpus import wordnet

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Key architechture

https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk

In [ ]:
def remove_adverbs(text):

  # Tokenize the text
  tokens = nltk.word_tokenize(text)

  # Use NLTK POS tagger to get parts of speech
  tagged_tokens = nltk.pos_tag(tokens)

  # Filter out adverbs (RB, RBR, RBS)
  filtered_text = [word for word, pos in tagged_tokens if pos not in ['RB', 'RBR', 'RBS']]

  # Join the filtered words back into a string
  result = ' '.join(filtered_text)

  return result

In [ ]:
def remove_contractions(text):

    contraction_dict = {"ain't": "is not", "aren't": "are not", "'cause": "because",
                        "can't": "cannot", "could've": "could have", "couldn't": "could not",
                        "didn't": "did not",  "doesn't": "does not", "don't": "do not",
                        "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                        "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                        "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                        "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
                        "I'll've": "I will have","I'm": "I am", "I've": "I have",
                        "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                        "i'll've": "i will have","i'm": "i am", "i've": "i have",
                        "isn't": "is not", "that's": "that is", "it'd": "it would",
                        "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                        "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                        "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                        "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
                        "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                        "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                        "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                        "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                        "she's": "she is", "should've": "should have", "shouldn't": "should not",
                        "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is",
                        "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                        "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                        "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have",
                        "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                        "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                        "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
                        "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                        "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                        "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is",
                        "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                        "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                        "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                        "y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                        "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

    def _get_contractions(contraction_dict):
        contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
        return contraction_dict, contraction_re

    def replace_contractions(text):
        contractions, contractions_re = _get_contractions(contraction_dict)
        def replace(match):
            return contractions[match.group(0)]
        return contractions_re.sub(replace, text)

    # replace contractions
    cleaned_text = replace_contractions(text)

    return cleaned_text

In [ ]:
def remove_pronouns(text):
  pattern = r"\b(I|me|my|myself|we|us|our|ours|ourselves|you|your|yours|yourself|yourselves|he|him|his|himself|she|her|hers|herself|it|its|itself|they|them|theirs|themselves|what|which|who|whom|whose|that|this|these|those)\b"
  filtered_text = re.sub(pattern, "", text, flags=re.IGNORECASE)
  return filtered_text

In [ ]:
def count_progressives(text):
  progressive_tenses = {
      "present progressive": r"\b(?:am|is|are|m|s|re)\s+(?!being)\w+ing\b",
      "past progressive":  r"\b(?:was|were)\s+(?!being)\w+ing\b",
      "future progressive": r"will be\s+(?!being)\w+ing\b",
      "present perfect progressive": r"(?<!\bwill\s)(?:has|have) been [a-zA-Z]+ing\b",
      "past perfect progressive": r"had been [a-zA-Z]+ing\b",
      "future perfect progressive": r"\bwill have been [a-zA-Z]+ing\b",
      "passive present progressive": r"(?:am|are|is) being \w+",
      "passive past progressive": r"(?:was|were) being \w+",
      "passive future progressive": r"will be being \w+",
      }

  progressive_counts = {tense: 0 for tense in progressive_tenses}

  # Prepare the text: regex counts in forms with only one whitspaces and without contractions
  # with no negatives or questions either
  clean_text = remove_adverbs(text)
  clean_text = remove_contractions(clean_text)
  clean_text = clean_text.replace('not', '')
  clean_text = remove_pronouns(clean_text)

  # Iterate through the text and count occurrences of progressive tenses
  for tense, pattern in progressive_tenses.items():
      progressive_counts[tense] = len(re.findall(pattern, clean_text))
      #print(re.findall(pattern, text))

  # Print the counts for each progressive tense
  #for tense, count in progressive_counts.items():
    #print(f"{tense}: {count}")

  return progressive_counts

In [ ]:
# Define the names of the new columns
tenses = {"present progressive" : [],
          "past progressive" : [],
          "future progressive" : [],
          "present perfect progressive" : [],
          "past perfect progressive" : [],
          "future perfect progressive" : [],
          "passive present progressive" : [],
          "passive past progressive" : [],
          "passive future progressive" : []
          }

text = """Certainly! Here are three examples for each of the specified verb tenses:

"present progressive":
- She is studying for her exams.
- They are playing soccer in the park.
- He is reading a book at the library.

"past progressive":
- She was cooking dinner when the guests arrived.
- They were watching a movie when the power went out.
- He was working on his car all afternoon.

"future progressive":
- She will be attending the conference next week.
- They will be traveling to Europe this summer.
- He will be starting his new job in September.

"present perfect progressive":
- She has been studying English for two years.
- They have been living in this city since 2010.
- He has been working on his project all day.

"past perfect progressive":
- She had been waiting for over an hour when he finally arrived.
- They had been studying together before the exam.
- He had been working at that company for ten years before retiring.

"future perfect progressive":
- She will have been working here for five years by next summer.
- They will have been living abroad for a decade by the time they return.
- He will have been saving money for his trip for six months by then.

"passive present progressive":
- The book is being read by many people around the world.
- The project is being completed by the team.
- The house is being renovated this year.

"passive past progressive":
- The report was being reviewed when the system crashed.
- The event was being organized by the volunteers.
- The problem was being addressed by the engineers.

"passive future progressive":
- The new policies will be being implemented by the end of the year.
- The improvements will be being made to the infrastructure in the coming months.
- The project will be being discussed at the next meeting."""

dict = count_progressives(text)

for tense, count in dict.items():
  print(f"{tense}: {count}")

present progressive: 3
past progressive: 3
future progressive: 3
present perfect progressive: 3
past perfect progressive: 3
future perfect progressive: 3
passive present progressive: 3
passive past progressive: 3
passive future progressive: 3


# Using the architechture

## Experiment

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1gCGbGxDMo9m3PZDrYtUz3S-MyVVilsD_svTfX5i5CAc/export?format=csv'
df = pd.read_csv(url)
df.head(5)

,l1,l2,Score,Story
0,0,NS,7,"the lady called the waiter over, when a bird r..."
1,1,C2,9,A lady sits outside a café with her dog and as...
2,3,C2,11,Bobby always loved birds and it hurt him to se...
3,3,B2,5,"The woman is in the cafe, she seats under the ..."
4,0,NS,12,Jane called to the waiter to order a coffee. O...


In [ ]:
# Define the names of the new columns
tenses = {"present progressive" : [],
          "past progressive" : [],
          "future progressive" : [],
          "present perfect progressive" : [],
          "past perfect progressive" : [],
          "future perfect progressive" : [],
          "passive present progressive" : [],
          "passive past progressive" : [],
          "passive future progressive" : []
          }

for text in df['Story']:
  # print(text)
  dict = count_progressives(text)
  for tense in tenses.keys():
    tenses[tense].append(dict[tense])

print(tenses)

{'present progressive': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0], 'past progressive': [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 2, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 0, 0, 0, 0, 1, 0, 1, 0, 2, 2, 0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 2, 0, 0, 1, 2, 0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 1, 1], 'future progressive': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'present perfect progressive': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
for tense in tenses.keys():
  df[tense] = tenses[tense]

df.head(10)

,l1,l2,Score,Story,present progressive,past progressive,future progressive,present perfect progressive,past perfect progressive,future perfect progressive,passive present progressive,passive past progressive,passive future progressive
0,0,NS,7,"the lady called the waiter over, when a bird r...",0,0,0,0,0,0,0,0,0
1,1,C2,9,A lady sits outside a café with her dog and as...,0,1,0,0,0,0,0,0,0
2,3,C2,11,Bobby always loved birds and it hurt him to se...,0,0,0,0,0,0,0,0,0
3,3,B2,5,"The woman is in the cafe, she seats under the ...",0,0,0,0,0,0,0,0,0
4,0,NS,12,Jane called to the waiter to order a coffee. O...,0,0,0,0,0,0,0,0,0
5,0,NS,12,All winter the goldfinch had lived happily in ...,0,0,0,0,0,0,0,0,0
6,2,C2,9,"On a brisk day in Autumn, Alex and Bryan were ...",0,1,0,0,1,0,0,0,0
7,0,NS,8,"On a cool fall day, two friends walk by a tree...",0,0,0,0,0,0,0,0,0
8,2,C2,8,Jane and her dog Luca went to coffee place in ...,0,0,0,0,0,0,0,0,0
9,0,NS,12,"On a sunny day last August, Hannah was out for...",0,1,0,0,0,0,0,0,0


In [ ]:
df_task3 = df
df_task3.to_csv('/content/drive/MyDrive/UNIVERSITY STUFF/ВКР/tenses_responses_task3.csv', index=False)

## TedTalks Corpus

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/10TjsyR1GlrAhM_t598iUbQtiIj28EOIn0uv9yjaoSx8/export?format=csv'
df_ted = pd.read_csv(url)
df_ted.head(5)

,l1,name,Topic,link,transcript
0,hebrew,Uri Hasson,neuroscience,https://www.youtube.com/watch?v=FDhlOovaGrI,Imagine that you invented a device that can re...
1,french,Sandrine Thuret,neuroscience,https://www.youtube.com/watch?v=B_tjKYvEziI,"Can we, as adults, grow new nerve cells? There..."
2,german,Gero Miesenboeck,neuroscience,https://www.youtube.com/watch?v=EZ_f3Fc0ZRA,I have a doppelganger. (Laughter) Dr. Gero is ...
3,chinese,Yejin Choi,ai,https://www.youtube.com/watch?v=SvBR0OGT5VI,So I'm excited to share a few spicy\nthoughts ...
4,swedish,Max Tegmark,ai,https://www.youtube.com/watch?v=2LRwvU6gEbA,"After 13.8 billion years\nof cosmic history, o..."


In [ ]:
def return_assigned_number(word, l_dict):
    for key, values in l_dict.items():
      if word in values:
        return key
    return None

l1_dict = {
    int(0): {"english"},
    int(1): {"afrikaans", "spanish", "italian", "portuguese", "slovenian"},
    int(2): {"armenian", "cantonese", "chinese", "danish", "dutch", "hindi", "igbo",
             "japanese", "kiswahili", "korean", "maltese", "nepali", "persian",
             "swahili", "tamil", "thai", "turkish"},
    int(3): {"arabic", "bulgarian", "croatian", "czech", "finnish", "french",
             "german", "greek", "hebrew", "kurdish", "norwegian", "polish", "russian",
             "serbian", "swedish", "ukranian", "yiddish"}
}

df_ted['l1_num'] = df_ted['l1'].apply(lambda x: return_assigned_number(x, l1_dict))

In [ ]:
df_ted.head(5)

,l1,name,Topic,link,transcript,l1_num
0,hebrew,Uri Hasson,neuroscience,https://www.youtube.com/watch?v=FDhlOovaGrI,Imagine that you invented a device that can re...,3
1,french,Sandrine Thuret,neuroscience,https://www.youtube.com/watch?v=B_tjKYvEziI,"Can we, as adults, grow new nerve cells? There...",3
2,german,Gero Miesenboeck,neuroscience,https://www.youtube.com/watch?v=EZ_f3Fc0ZRA,I have a doppelganger. (Laughter) Dr. Gero is ...,3
3,chinese,Yejin Choi,ai,https://www.youtube.com/watch?v=SvBR0OGT5VI,So I'm excited to share a few spicy\nthoughts ...,2
4,swedish,Max Tegmark,ai,https://www.youtube.com/watch?v=2LRwvU6gEbA,"After 13.8 billion years\nof cosmic history, o...",3


In [ ]:
# Define the names of the new columns
tenses = {"present progressive" : [],
          "past progressive" : [],
          "future progressive" : [],
          "present perfect progressive" : [],
          "past perfect progressive" : [],
          "future perfect progressive" : [],
          "passive present progressive" : [],
          "passive past progressive" : [],
          "passive future progressive" : []
          }

for text in df_ted['transcript']:
  # print(text)
  dict = count_progressives(text)
  for tense in tenses.keys():
    tenses[tense].append(dict[tense])

print(tenses)

{'present progressive': [46, 6, 3, 9, 15, 14, 10, 23, 7, 2, 8, 30, 20, 1, 7, 14, 14, 4, 8, 1, 1, 1, 3, 24, 9, 10, 7, 8, 11, 6, 1, 9, 10, 10, 21, 10, 5, 4, 4, 21, 18, 16, 21, 16, 17, 13, 2, 18, 20, 22, 4, 7, 5, 40, 15], 'past progressive': [2, 3, 1, 0, 0, 2, 1, 0, 2, 2, 1, 1, 3, 1, 0, 1, 0, 3, 1, 1, 4, 0, 12, 0, 16, 8, 4, 19, 10, 0, 4, 3, 2, 6, 1, 3, 0, 1, 7, 4, 12, 7, 1, 5, 34, 9, 1, 2, 2, 1, 12, 0, 0, 1, 0], 'future progressive': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'present perfect progressive': [0, 1, 0, 3, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2], 'past perfect progressive': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'futu

In [ ]:
for tense in tenses.keys():
  df_ted[tense] = tenses[tense]

df_ted.head(5)

,l1,name,Topic,link,transcript,l1_num,present progressive,past progressive,future progressive,present perfect progressive,past perfect progressive,future perfect progressive,passive present progressive,passive past progressive,passive future progressive
0,hebrew,Uri Hasson,neuroscience,https://www.youtube.com/watch?v=FDhlOovaGrI,Imagine that you invented a device that can re...,3,46,2,0,0,0,0,0,0,0
1,french,Sandrine Thuret,neuroscience,https://www.youtube.com/watch?v=B_tjKYvEziI,"Can we, as adults, grow new nerve cells? There...",3,6,3,0,1,0,0,0,0,0
2,german,Gero Miesenboeck,neuroscience,https://www.youtube.com/watch?v=EZ_f3Fc0ZRA,I have a doppelganger. (Laughter) Dr. Gero is ...,3,3,1,0,0,0,0,0,0,0
3,chinese,Yejin Choi,ai,https://www.youtube.com/watch?v=SvBR0OGT5VI,So I'm excited to share a few spicy\nthoughts ...,2,9,0,0,3,0,0,0,0,0
4,swedish,Max Tegmark,ai,https://www.youtube.com/watch?v=2LRwvU6gEbA,"After 13.8 billion years\nof cosmic history, o...",3,15,0,0,2,0,0,0,0,0


In [ ]:
df_ted.to_csv('/content/drive/MyDrive/UNIVERSITY STUFF/ВКР/ted_talks_corpus_tenses.csv', index=False)

## Book Corpus

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1wopAoFHAc41lcWp0__TdKSc3I6OzSbn_VnB6n-nieP0/export?format=csv'
df = pd.read_csv(url)

# Select columns from 6 (included) to 9 (excluded)
df_books = df.iloc[:, 5:9]  # Remember Python slicing excludes the end index
df_books = df_books.iloc[:37]

new_names = {'l1.1': 'l1'}
df_books = df_books.rename(columns=new_names)

df_books.head(5)

,l1,author,book
0,italian,Francesca Marciano,Marciano Francesca - The Other Language Storie...
1,italian,Francesca Marciano,Marciano Francesca - Rules of the Wild.txt
2,italian,Francesca Marciano,Marciano Francesca - Animal Spirit Stories.txt
3,italian,Francesca Marciano,Marciano Francesca - The Presence of Men.txt
4,afrikaans,Andre Brink,Andre Brink - Rumours Of Rain.txt


In [ ]:
def return_assigned_number(word, l_dict):
    for key, values in l_dict.items():
      if word in values:
        return key
    return None

l1_dict = {
    int(0): {"english"},
    int(1): {"afrikaans", "spanish", "italian", "portuguese", "slovenian"},
    int(2): {"armenian", "cantonese", "chinese", "danish", "dutch", "hindi", "igbo",
             "japanese", "kiswahili", "korean", "maltese", "nepali", "persian",
             "swahili", "tamil", "thai", "turkish"},
    int(3): {"arabic", "bulgarian", "croatian", "czech", "finnish", "french",
             "german", "greek", "hebrew", "kurdish", "norwegian", "polish", "russian",
             "serbian", "swedish", "ukranian", "yiddish"}
}

df_books['l1_num'] = df_books['l1'].apply(lambda x: return_assigned_number(x, l1_dict))

In [ ]:
df_books.head(5)

,l1,author,book,l1_num
0,italian,Francesca Marciano,Marciano Francesca - The Other Language Storie...,1
1,italian,Francesca Marciano,Marciano Francesca - Rules of the Wild.txt,1
2,italian,Francesca Marciano,Marciano Francesca - Animal Spirit Stories.txt,1
3,italian,Francesca Marciano,Marciano Francesca - The Presence of Men.txt,1
4,afrikaans,Andre Brink,Andre Brink - Rumours Of Rain.txt,1


In [ ]:
# Define the names of the new columns
tenses = {"present progressive" : [],
          "past progressive" : [],
          "future progressive" : [],
          "present perfect progressive" : [],
          "past perfect progressive" : [],
          "future perfect progressive" : [],
          "passive present progressive" : [],
          "passive past progressive" : [],
          "passive future progressive" : []
          }
chars = []
words = []

folder_path = '/content/drive/MyDrive/UNIVERSITY STUFF/ВКР/books/'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
  # Check if it's a text file (optional)
  if filename.endswith(".txt"):
    file_path = os.path.join(folder_path, filename)

    # Open the file for reading
    with open(file_path, 'r') as f:
      # Process the file content here
      file_content = f.read()

      dict = count_progressives(text=file_content)

      for tense in tenses.keys():
        tenses[tense].append(dict[tense])

      # wordcount
      num_words = len(file_content.split())
      # Count the number of characters (excluding whitespaces)
      num_chars = len(file_content.replace(" ", ""))

      chars.append(num_chars)
      words.append(num_words)

print(tenses)

{'present progressive': [141, 294, 99, 23, 414, 244, 351, 234, 222, 288, 262, 221, 85, 108, 220, 69, 186, 278, 41, 102, 426, 125, 145, 372, 637, 51, 155, 14, 85, 76, 95, 422, 134, 460, 146, 169, 173], 'past progressive': [375, 345, 342, 70, 87, 330, 535, 268, 431, 288, 414, 518, 61, 213, 177, 65, 308, 193, 39, 261, 509, 413, 333, 821, 933, 145, 237, 63, 393, 230, 73, 633, 224, 749, 271, 239, 470], 'future progressive': [0, 2, 1, 0, 4, 0, 0, 2, 0, 1, 4, 1, 0, 1, 3, 0, 0, 0, 0, 0, 11, 1, 2, 4, 3, 1, 3, 0, 6, 6, 1, 3, 2, 13, 1, 6, 4], 'present perfect progressive': [17, 9, 1, 2, 15, 4, 5, 9, 7, 13, 6, 5, 8, 5, 5, 1, 3, 3, 1, 5, 21, 2, 3, 10, 19, 2, 17, 1, 7, 4, 13, 16, 4, 13, 2, 10, 5], 'past perfect progressive': [33, 28, 25, 6, 1, 9, 19, 35, 25, 16, 11, 5, 7, 8, 12, 0, 3, 2, 0, 22, 46, 22, 34, 46, 81, 12, 22, 5, 48, 20, 1, 25, 11, 27, 24, 23, 71], 'future perfect progressive': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
df_books['words'] = words
df_books['characters'] = chars

for tense in tenses.keys():
  df_books[tense] = tenses[tense]

df_books.head(5)

,l1,author,book,l1_num,words,characters,present progressive,past progressive,future progressive,present perfect progressive,past perfect progressive,future perfect progressive,passive present progressive,passive past progressive,passive future progressive
0,italian,Francesca Marciano,Marciano Francesca - The Other Language Storie...,1,91708,422238,141,375,0,17,33,0,1,3,0
1,italian,Francesca Marciano,Marciano Francesca - Rules of the Wild.txt,1,93664,419728,294,345,2,9,28,0,3,8,0
2,italian,Francesca Marciano,Marciano Francesca - Animal Spirit Stories.txt,1,62172,290671,99,342,1,1,25,0,1,1,0
3,italian,Francesca Marciano,Marciano Francesca - The Presence of Men.txt,1,16954,77924,23,70,0,2,6,0,1,1,0
4,afrikaans,Andre Brink,Andre Brink - Rumours Of Rain.txt,1,97176,409009,414,87,4,15,1,0,2,1,0


# Save

In [ ]:
df_books.to_csv('/content/drive/MyDrive/UNIVERSITY STUFF/ВКР/books_corpus_tenses.csv', index=False)